## Notebook: Contextual Anomaly Detection (CAD) Device Creation

This is the second of a series of three notebooks that show how the Contextual Anomaly Detection (CAD) Accelerator can be used to train and deploy a prediction interval model into Monitor using the Model Factory service endpoints.
1.   Training: cookbooks/contextual_anomaly_train.ipynb
2.  Monitor Device Creation: cookbooks/contextual_anomaly_create_device.ipynb
3.   Model Deployment: cookbooks/contextual_anomaly_deploy_model.ipynb


### CAD Description

The CAD training job produces three prediction interval models that capture the normal operation (non-anomalous) behaviour of a given target variable based on a set of input features. 

These are point estimate multivariate regression machine learning models (denoted as base regressor) enhanced with conformal prediction statistical wrappers to produce a lower and upper bound that contain the target variable with probability 95% under normal operation (non-anomalous) conditions. Therefore, the probability of observing a target variable outside of the provided interval is 5% under normal operation conditions.

### Wind Turbine Dataset Description: 

In this notebook we use the CAD Accelerator to learn a prediction interval model that covers the normal operation behaviour of the Average Reactive Power of a Wind turbine asset with probability 95%. We consider 4 input features to predict the target variable. These are Average Active Power, Average Generator Bearing 1 Temperature, Average Generator Bearing 2 Temperature, Average Wind Speed.

### Device Creation Notebook Description

This notebook shows an example of how the Model Factory Service can be used to create a monitor device needed before deploying the trained model. The device created in Monitor will contain as metrics the different variables of the dataset (features,target and time stamp).


This is the SECOND notebook of our tutorial and should be run after 'cookbooks/contextual_anomaly_train.ipynb' since it requires the model specification file generated at the end of the CAD training notebook.





<a id='notebook_workflow'></a>
### Notebook Workflow
- [Imports](#imports)
- [Load KPI specification file (yaml)](#load_kpiyaml)
- [Load Model Factory config file (yaml)](#load_mfyaml)
- [Load Model specification file (yaml)](#load_modelspecyaml)
- [Prepare Payload to Create Monitor Device](#prepare_payload)
- [Post Create Device Job ](#create_device_post)
- [Request job log and summary](#job_log)

<a id='imports'></a>
### Imports

In [1]:
import requests
import yaml
import pandas as pd

<a id='load_kpiyaml'></a>
### Load KPI specification file (yaml)


This file is a dictionary containing:

    - asset_id_column: column corresponding to asset id
    - data_name: train dataset file name
    - device_description: device description (optional)
    - mas_device_name: monitor device name (needed for device creation and/or deployment)
    - feature_columns: feature columns name as in first row of dataset csv file separated by ',' (e.g., P_avg,Rs_avg,Gb1t_avg,Ws_avg)
    - feature_names: feature columns interpretable names separated by ',' (same order as in feature_columns)
    - target_columns: target column name as in first row of dataset
    - target_names: target column interpretable name
    - timestamp_column: time stamp column name as in first row of dataset
    - timestamp_format: '%m/%d/%Y %H:%M'
    - inference_data_name: test dataset file name (optional for inference)
    - feature_map: dictionary mapping feature columns to descriptions (optional)

In [2]:
input_file_name = "../config/Pavg_kpi.yml"

with open(input_file_name, 'r') as file:
    input_data = yaml.safe_load(file)

print('KPI specification file: ')
print(input_data)

KPI specification file: 
{'asset_id_column': 'Wind_turbine_name', 'data_name': 'Wind_Turbine_train.csv', 'device_description': 'Wind Turbine', 'mas_device_name': 'Wind_Turbine_Test_1', 'feature_columns': 'P_avg,Gb1t_avg,Gb2t_avg,Ws_avg', 'feature_names': 'P_average,Gblt_average,Gb2t_average,Ws_average', 'target_columns': 'Rs_avg', 'target_names': 'Rs_avg', 'timestamp_column': 'Date_time', 'timestamp_format': '%m/%d/%Y %H:%M', 'inference_data_name': 'Wind_Turbine_test.csv', 'feature_map': {'P_avg': 'Average Active Power', 'Rs_avg': 'Average Reactive Power', 'Gb1t_avg': 'Average Generator Bearing 1 Temperature', 'Gb2t_avg': 'Average Generator Bearing 2 Temperature', 'Gost_avg': 'Average Generator Outer Stator Temperature', 'Git_avg': 'Average Generator Inner Stator Temperature', 'Yt_avg': 'Average Yaw System Temperature', 'Ot_avg': 'Average Outdoor Temperature', 'Ws_avg': 'Average Wind Speed', 'Wa_avg': 'Average Wind Direction'}}


<a id='load_mfyaml'></a>
### Load Model Factory config file (yaml)


This file is a dictionary containing:

    - endpoint_url: <ACTION: Replace with Model Factory endpoint URL>
    - train_recipe_endpoint: recipe/supervised-anomaly (DON'T CHANGE)
    - deploy_recipe_endpoint: deployment/monitor/model/create (DON'T CHANGE)
    - create_device_recipe_endpoint: deployment/monitor/device/create (DON'T CHANGE)

In [3]:
model_factory_config_file_name = "../config/model_factory_config.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'endpoint_url': 'http://127.0.0.1:8000/ibm/modelfactory/service/', 'train_recipe_endpoint': 'recipe/supervised-anomaly', 'deploy_recipe_endpoint': 'deployment/monitor/model/create', 'create_device_recipe_endpoint': 'deployment/monitor/device/create'}


<a id='load_modelspecyaml'></a>
### Load Model specification file (yaml)

This is the .yaml file saved in the last step of the contextual_anomaly_train.ipynb notebook. It is a dictionary containing the following information:

    - onnx_model_uri : Model mlflow uri provided in the CAD training summary
    - train_job_id : Model Factory CAD training job id
    - mas_device_name : Name of the Monitor Device

In [4]:
input_file_name = "../config/model_info.yml"

with open(input_file_name, 'r') as file:
    model_info_data = yaml.safe_load(file)

print(model_info_data)

{'mas_device_name': 'Wind_Turbine_Test_999', 'onnx_model_uri': 's3://testdataupload/48/f5b935499e3e488cbd83235cb376343b/artifacts/PI_model_prefit_LGBM1_best.onnx', 'train_job_id': '932740d6-9ac6-4bce-8adf-9db53fe146ae'}


<a id='prepare_payload'></a>
### Prepare Payload to Create Monitor Device

The create device payload requires the following files:

- iot_credentials: This file is located in config/iot_credentials.yaml and is a dictionary containing the following information
    - IOT_URL: <INSERT>
    - IOT_API_KEY: <INSERT>
    - IOT_API_TOKEN: <INSERT>
    - model_api: /api/v0002/pipeline/models
    - MAS_URL: <INSERT>
    - X-api-key: <INSERT>
    - X-api-token: <INSERT>
    - mam_user_email: <INSERT>
    - tenantId: <INSERT>


- data_file: This file is a .csv file containing a single row of data (i.e., first two rows of ../data/Wind_Turbine_train.csv). In this example we use the file located in '../data/Wind_Turbine_Asset_Data.csv'

- assetmodel_file: This file is a .csv file containing 4 columns corresponding to dimension (name of variable/feature), type, label and columntype (following Monitor convention). In this example we use the file located in '../data/Wind_Turbine_Asset_Model.csv'

- device_type_name: Name of mas device (this will be obtained from the Model Specification file, [see above](#load_modelspecyaml) )

- device_type_description: Text description of the MAS device

- metric_columns: The metric associated with the device and available as columns in the data_file.

- timestamp_column: Time stamp column name (this will be obtained from the KPI specification file, [see above](#load-kpi-specification-file-yaml)).

- assetid_column: Asset id column name (this will be obtained from the KPI specification file, [see above](#load-kpi-specification-file-yaml)).

In [5]:
'''
Credential File Path and Name
'''

credentials_file_path = "../config/iot_credentials_beta.yaml"
credentials_file_name = "iot_credentials_beta.yaml"

'''
Data File Path and Name
'''

data_file_create_device_path = "../data/Wind_Turbine_Asset_Data.csv"
data_file_create_device_name = "Wind_Turbine_Asset_Data.csv"

'''
assetmodel File Path and Name
'''

assetmodel_file_create_device_path = "../data/Wind_Turbine_Asset_Model.csv"
assetmodel_file_create_device_name = "Wind_Turbine_Asset_Model.csv"


'''
Device Type Name and Description
'''

device_type_name = model_info_data['mas_device_name']
device_type_description= 'Wind Turbine Device'

'''
Columns
'''

metric_columns = pd.read_csv(assetmodel_file_create_device_path)['dimension'].unique()
metric_columns = ",".join(metric_columns)

timestamp_column = input_data['timestamp_column']
assetid_column = input_data['asset_id_column']

In [6]:
payload = {}
payload["device_type_name"] = device_type_name
payload["device_type_description"] = device_type_description
payload["metric_columns"] = metric_columns
payload["timestamp_column"] = timestamp_column
payload["assetid_column"] = assetid_column


files = [
    ("iot_credentials", (credentials_file_name, open(credentials_file_path), "text/csv")),
    ("data_file", (data_file_create_device_name, open(data_file_create_device_path), "text/csv")),
    ("assetmodel_file", (assetmodel_file_create_device_name, open(assetmodel_file_create_device_path), "text/csv")),

]

In [7]:
print(payload)

{'device_type_name': 'Wind_Turbine_Test_999', 'device_type_description': 'Wind Turbine Device', 'metric_columns': 'P_avg,Rs_avg,Gb1t_avg,Gb2t_avg,Gost_avg,Git_avg,Yt_avg,Ot_avg,Ws_avg,Wa_avg', 'timestamp_column': 'Date_time', 'assetid_column': 'Wind_turbine_name'}


<a id='create_device_post'></a> 
### Post Create Device Job 

In [8]:
endpoint_url = model_factory_config["endpoint_url"]
create_device_url = endpoint_url + model_factory_config["create_device_recipe_endpoint"]
print(create_device_url)

http://127.0.0.1:8000/ibm/modelfactory/service/deployment/monitor/device/create


In [9]:
headers = {
  'accept': 'application/json'
}
response = requests.post(create_device_url, data=payload, files=files, headers=headers)

post_r_json = response.json()
print(post_r_json)
job_id = post_r_json['job_id']
print('job id:',job_id )

{'job_id': 'b0dbbda5-c70f-4f7d-b996-e53d1a4e860b', 'message': 'Monitor device creation job b0dbbda5-c70f-4f7d-b996-e53d1a4e860b submitted for creation.', 'status': 'INITIALIZING'}
job id: b0dbbda5-c70f-4f7d-b996-e53d1a4e860b


<a id='job_log'></a> 
### Request job log and summary

In [15]:
logs = requests.get(endpoint_url + "log/" + job_id, headers=headers)
print('LOGS :: ')
if "logs" in logs.json():
    print(logs.json()["logs"])
else:
    print(logs.json())

summary = requests.get(endpoint_url + "summary/" + job_id, headers=headers)
print('SUMMARY :: ')
print(summary.json())

LOGS :: 
recipe_file_path /tmp/ray/session_2023-11-27_21-19-44_821413_1/runtime_resources/working_dir_files/_ray_pkg_a66b93882930c2ef/Recipe.create
entrypoint_path /tmp/ray/session_2023-11-27_21-19-44_821413_1/runtime_resources/working_dir_files/_ray_pkg_a66b93882930c2ef/main.py
2023-11-27 21:20:25,424	INFO worker.py:1329 -- Using address 172.63.0.3:6379 set in the environment variable RAY_ADDRESS
2023-11-27 21:20:25,424	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 172.63.0.3:6379...
2023-11-27 21:20:25,452	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 172.63.0.3:8265 
(run_recipe_on_ray pid=381) 2023/11/27 21:20:28 INFO mlflow.tracking.fluent: Experiment with name 'b0dbbda5-c70f-4f7d-b996-e53d1a4e860b' does not exist. Creating a new experiment.
(run_recipe_on_ray pid=381) mlflow.experiment.name: b0dbbda5-c70f-4f7d-b996-e53d1a4e860b
(run_recipe_on_ray pid=381) 2023/11/27 21:20:29 WARNING mlflow.utils.git_utils: Failed to import Git (the